## Bad Coils

In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif

# -------- Load file --------
path = r"bucket_coil_signal_averages_allsignals_phasewise_rollingmode_reshaped_with_clusters_bad.csv"
df = pd.read_csv(path)

# -------- Identify cluster column --------
cluster_col = "cluster_label"
y = df[cluster_col]

# -------- Select only signal columns containing "_Phase" --------
signal_cols = [c for c in df.columns if "_Phase" in c]

# Keep only numeric signal columns
X = df[signal_cols].select_dtypes(include=[np.number])

# -------- Drop rows having NaN in any signal column --------
X = X.dropna()
y = y.loc[X.index]

# -------- ANOVA F-test --------
F_values, p_values = f_classif(X, y)

results = pd.DataFrame({
    "Feature": X.columns,
    "F_value": F_values,
    "p_value": p_values
}).sort_values("F_value", ascending=False)

print(results["Feature"].head(15))

136       Stand 4 Bottom Current Feedback_Phase2
43           Stand 4 Top Current Feedback_Phase2
106    Stand 1-3 Solution System Pressure_Phase2
135       Stand 4 Bottom Current Feedback_Phase1
105    Stand 1-3 Solution System Pressure_Phase1
107    Stand 1-3 Solution System Pressure_Phase3
137       Stand 4 Bottom Current Feedback_Phase3
7         Tension Reel Calculated Tension_Phase2
6         Tension Reel Calculated Tension_Phase1
8         Tension Reel Calculated Tension_Phase3
138       Stand 1 Bottom Current Feedback_Phase1
149      Stand 4 Solution System Pressure_Phase3
44           Stand 4 Top Current Feedback_Phase3
139       Stand 1 Bottom Current Feedback_Phase2
42           Stand 4 Top Current Feedback_Phase1
Name: Feature, dtype: object


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  3   4  10  16  27  28  54  55  79  85  90  91 130] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


## Good Coils

In [12]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif

# -------- Load file --------
path = r"bucket_coil_signal_averages_allsignals_phasewise_rollingmode_reshaped_with_clusters.csv"
df = pd.read_csv(path)

# -------- Identify cluster column --------
cluster_col = "cluster_label"
y = df[cluster_col]

# -------- Select only signal columns containing "_Phase" --------
signal_cols = [c for c in df.columns if "_Phase" in c]

# Keep only numeric signal columns
X = df[signal_cols].select_dtypes(include=[np.number])

# -------- Drop rows having NaN in any signal column --------
X = X.dropna()
y = y.loc[X.index]

# -------- ANOVA F-test --------
F_values, p_values = f_classif(X, y)

results = pd.DataFrame({
    "Feature": X.columns,
    "F_value": F_values,
    "p_value": p_values
}).sort_values("F_value", ascending=False)

print(results["Feature"].head(15))

106    Stand 1-3 Solution System Pressure_Phase2
148      Stand 4 Solution System Pressure_Phase2
133           Stand 3-4 Tension Reference_Phase2
105    Stand 1-3 Solution System Pressure_Phase1
149      Stand 4 Solution System Pressure_Phase3
74          Stand 4 - Operator Side Force_Phase3
107    Stand 1-3 Solution System Pressure_Phase3
132           Stand 3-4 Tension Reference_Phase1
53         Stand 2 Total Bending Feedback_Phase3
72          Stand 4 - Operator Side Force_Phase1
73          Stand 4 - Operator Side Force_Phase2
62            Stand 4 Thread Gap Setpoint_Phase3
60            Stand 4 Thread Gap Setpoint_Phase1
61            Stand 4 Thread Gap Setpoint_Phase2
137       Stand 4 Bottom Current Feedback_Phase3
Name: Feature, dtype: object


C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  3   4  10  16  27  28  54  55  79  84  85  90  91 129 130] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import f_classif
import re

# Load
path = r"bucket_coil_signal_averages_allsignals_phasewise_rollingmode_reshaped.csv"
df = pd.read_csv(path)

signal_cols = [c for c in df.columns if "_Phase" in c]

# Buckets to compare
target_buckets = [6, 5, 1, 8, 12, 20, 10, 2]

# Filter only required buckets
df = df[df["Bucket_ID"].isin(target_buckets)]

bucket_top = {}
top_n = 30   # top 30 signals per bucket

def get_base_name(col):
    return re.sub(r"_Phase\d+", "", col)

for b in target_buckets:
    df_b = df.copy()
    df_b["target"] = (df_b["Bucket_ID"] == b).astype(int)

    X = df_b[signal_cols].select_dtypes(include=[np.number]).dropna()
    y = df_b.loc[X.index, "target"]

    F_vals, _ = f_classif(X, y)

    tmp = pd.DataFrame({
        "Feature": X.columns,
        "Base": [get_base_name(c) for c in X.columns],
        "F_value": F_vals
    }).sort_values("F_value", ascending=False)

    # keep top N base names (unique)
    top_list = []
    for _, row in tmp.iterrows():
        if row["Base"] not in top_list:
            top_list.append(row["Base"])
        if len(top_list) == top_n:
            break

    bucket_top[b] = top_list

# Find common signals across all buckets (based on base names)
common_signals = set(bucket_top[target_buckets[0]])
for b in target_buckets[1:]:
    common_signals = common_signals.intersection(bucket_top[b])

print("Common top signals across buckets (phase removed):")
print(common_signals)
print()

print("Top 30 base signals in each bucket:")
for b in target_buckets:
    print("\nBucket", b)
    print(bucket_top[b])


Common top signals across buckets (phase removed):
{'Stand 2 Top Current Feedback', 'Tension Reel Calculated Tension', 'Stand 1 Run Gap Setpoint', 'Stand 3 Run Gap Setpoint'}

Top 30 base signals in each bucket:

Bucket 6
['Stand 1 Run Gap Setpoint', 'Stand 1 Predicted Run Force', 'Stand 4 Thread Gap Setpoint', 'Stand 3 Run Gap Setpoint', 'Stand 2 Gap Thread Offset', 'Stand 3 - Operator Side Force', 'Stand 1 Gap Thread Offset', 'Stand 1 Bottom Current Feedback', 'Stand 2 Drive Speed Feedback', 'Stand 1 Roll Force Increase Limit (based on predicted run force)', 'Stand 4 DS Total Bending Feedback', 'X1 Gauge Deviation', 'Ramp Greater Than Thread', 'Stand 3 Gap Thread Offset', 'AGC GE Feedforward Hardness Number', 'Tension Reel Calculated Tension', 'Stand 4 - Operator Side Force', 'Stand 4 OS Total Bending Feedback', 'AGC Alex Dynamic Feedforward Hardness Number', 'Stand 2 Bottom Current Feedback', 'Stand 2 Gap Stick Offset', 'Stand 2 Top Current Feedback', 'Stand 3 Drive Speed Feedback',

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  3   4  10  16  27  28  54  55  79  84  85  90  91 129 130] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: Features [  3   4  10  16  27  28  54  55  79  84  85  90  91 129 130] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: UserWarning: F